<a href="https://colab.research.google.com/github/ArielIvan1981/ProyectoFinalCertificacion/blob/main/Analiza_Contenido_Sensible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalo las librerias a utilizar, usaremos BeautifulSoup, Requests y el Parser (el parser sirve para extraer la informacion de html) usaremos lxml

In [20]:
#pip install requests

In [21]:
#pip install lxml #usamos el parser lxml para obtener data del codigo html

In [22]:
#pip install bs4

Llamamos a las librerias

In [23]:
from bs4 import BeautifulSoup
import requests
import termcolor
import os
os.system('color')

32512

Le pedimos al usuario que ingrese por teclado la url a analizar, esto se hara manualmente en un principio pero luego se buscara la manera que se haga de forma automatica

In [24]:
website=input("Ingrese enlace a analizar: ")

Ingrese enlace a analizar: https://www.infobae.com/sociedad/policiales/2022/07/29/misterio-por-el-crimen-de-un-gendarme-en-rosario-encontraron-su-cuerpo-en-el-fondo-de-un-aljibe-y-hay-sospechas-sobre-su-ex-pareja/


Usamos requests para mandar una solicitud al servidor y obtener el codigo html de la pagina web y lo asignamos a la variable result, me muestra la respuesta de la pagina a la solicitud

In [25]:
result=requests.get(website)

A continuacion obtenemos el texto del resultado

In [26]:
content=result.text

Creamos una variable soup, nos permitira localizar elementos en una pagina web. En soup ya tenemos la pagina en codigo html

In [27]:
soup=BeautifulSoup(content, 'lxml')

Creamos la variable title para obtener el titular de la noticia. Para ello, buscamos el box html donde suele estar el titulo en los principales portales. Si no estamos trabajando con ninguno de ellos, usamos la etiqueta h1 del html.

In [28]:
title=None
def find_title(x):
  '''Funcion para encontrar el titulo de
  la noticia, dependiendo el diseno html del
  portal. Argumento "x" es
  un objeto soup'''
  box=None
  global title
  if 'pagina12' in website:
    box=x.find('article', class_= 'article-full section')
  elif 'infobae' in website:
    box=x.find('div', class_='article-header')
  elif 'lanacion' in website:
    box=x.find('div', class_='lay --apertura')
  elif 'clarin' in website:
    box=x.find('div',
                  class_='entry-title col-lg-6 col-md-12 col-sm-12 col-xs-12')
  elif 'lavoz' in website:
    box=x.find('div', class_='px2 story-headline')
  elif 'laizquierdadiario' in website:
    box=x.find('div', class_='header-articulo')
  else:
    title=x.find('h1').get_text()

  if box is not None:
    title=box.find('h1').get_text()


In [29]:
find_title(soup)

Realizamos lo mismo para el subtitulo



In [30]:
subtitle=None
def find_subtitle(x):
  '''Funcion para encontrar el subtitulo de
  la noticia, dependiendo el diseno html del
  portal. Argumento "x" es
  un objeto soup'''  
  global subtitle
  if 'pagina12' in website:
    subtitle=x.find('h3').get_text()
  elif 'infobae' in website:
    subtitle=x.find('h2').get_text()
  elif 'lanacion' in website:
    subtitle=x.find('h2').get_text()
  elif 'laizquierdadiario' in website:
    subtitle=x.find('p').get_text()
  elif 'lavoz' in website:
    subtitle=x.find('div', class_='story-subtitle').get_text()
  elif 'clarin' in website:
    subtitle=x.find('div', class_='bajada').get_text()
  elif 'quorum.gt' in website:
    subtitle=x.find('div', class_='post-excerpt').get_text()
  else:
    subtitule=x.find('h2').get_text()

In [31]:
find_subtitle(soup)

Buscamos el contenido del texto. Al igual que con el titulo, buscamos en el box donde suele estar el parrafo en los principales portales, luego extraemos el texto con la etiqueta 'p' de html.

In [32]:
parrafo=None
def find_parrafo(x):
  ''' Funcion para encontrar el contenido de la noticia
  dependiendo el diseno html del portal. Argumento "x" es
  un objeto soup'''
  box=None
  global parrafo
  if 'pagina12' in website:
    box=x.find('article', class_='article-main-content article-text')
  elif 'infobae' in website:
    box=x.find('div', class_='nd-body-article')
  elif 'lanacion' in website:
    box=x.find('div', class_='col-12')
  elif 'laizquierdadiario' in website:
    box=x.find('div', class_='col-md-10 offset-md-1')
  elif 'lavoz' in website:
    box=x.find('section')
  elif 'clarin' in website:
    box=x.find('div', class_='body-nota')
  else:
    parrafo=x.find('article').get_text()
  
  if box is not None:
    parrafo=box.find('p').get_text()

In [33]:
#Intentamos encontrar el parrafo con la funcion find_parrafo. 
#Si no lo encuentra, se imprime 'No se encontro parrafo', y esa variable continuara siendo None.
try:
  find_parrafo(soup)
except:
  print('No se encontro parrafo')

Creamos la variable texto_a_analizar para ir concatenenado titulo, subtitulo y contenido de la noticia a analizar

In [34]:
texto_a_analizar = f"{title} \n{subtitle} \n{parrafo}"

In [35]:
ls_words = texto_a_analizar.split()

Creamos una lista con las palabras clave que queremos buscar en el texto

In [36]:
ls_triggerwords= ['sangre', 'mató', 'mataron', 'asesinato', 
                  'fallecido', 'muerto', 'asesinaron', 'asesinó', 
                  'cadaver', 'sicarios', 'asesino', 'tiro', 
                  'dispararon', 'ejecutaron', 'ejecución', 'muertos',
                  'accidente', 'víctima', 'violación', 
                  'chocaron', 'exterminio', 'crimen', 'decapitaron']

Creamos la funcion para evaluar si en la noticia se encuentra alguna de la palabras o etiquetas que determinaran si el contenido de la noticia es sensible o no.

In [37]:
def find_trigger_words(x):
  global ls_triggerwords

  bandera = 0
  for w in x:
    if w in ls_triggerwords:
      bandera = 1
      break
    else:
      bandera = 0

  if bandera == 1:
    print(termcolor.colored('Precaucion Noticia con contenido sensible', 'red'))
  else:
    print(termcolor.colored('Noticia sin contenido sensible', 'green'))

In [38]:
find_trigger_words(ls_words)

Precaucion Noticia con contenido sensible
